###Implement support vector machine (SVM) classifier for the multi-class classification task. You can use one vs one and one vs all multiclass coding methods to create binary SVM models. Implement the SMO algorithm for the evaluation of the training parameters of SVM such as Lagrange multipliers. You can use holdout approach (70, 10, 20%) for evaluating the performance of the classifier. The dataset (data5.mat) contains 72 features and the last column is the output (class labels). Evaluate individual accuracy and overall accuracy. (Packages such as keras, tensorflow, pytorch for python and MATLAB deep learning toolbox etc. are not allowed)

In [1]:
#import all the necessory libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from mpl_toolkits import mplot3d
import scipy.io
import math

In [13]:
root="/content/drive/MyDrive/NNFL_Assignment2/data5.mat"
data=scipy.io.loadmat(os.path.join(root))
data = data['x']
df = pd.DataFrame(data)
data = np.array(df)
np.random.shuffle(data)
print((data.shape))


(2148, 73)


In [3]:
def split_train_test(dat,trainperc):
  train=dat[:round(dat.shape[0]*trainperc/100),:]
  test=dat[round(dat.shape[0]*trainperc/100):,:]
  return train , test


In [4]:
def split_x_y(data):
  x=np.delete(data,-1,1)
  x=normalize(x)
  y=data[:,-1]
  for insts in range(y.shape[0]):
    if y[insts]==0:
      y[insts]=-1
  return x,y

In [5]:
def normalize(data):
    for i in range(data.shape[1]):
        temp=data[:,i]
        data[:,i]=(temp-np.mean(temp))/np.std(temp)
    return data

In [6]:
# #setting up the data into train,test,valid,test and initializing parameters
# train,test=split_train_test(split_classes(data),80)
# train,valid=split_train_test(split_classes(train),87.5)
# x_train,y_train=split_x_y(train)
# y_train=onehottocls(y_train)
# x_valid,y_valid=split_x_y(valid)
# y_valid=onehottocls(y_valid)
# x_test,y_test=split_x_y(test)
# y_test=onehottocls(y_test)
# print(y_train.shape)
# print(x_valid.shape)
# print(y_valid.shape)
# print(x_test.shape)
# print(y_test.shape)

In [7]:
# train,test=split_train_test(data,80)
# train,valid=split_train_test(train,87.5)
# x_train,y_train=split_x_y(train)
# x_valid,y_valid=split_x_y(valid)
# x_test,y_test=split_x_y(test)

In [27]:
def SVMtrain(data,c,bound,iter):
  train,test=split_train_test(data,80)
  train,valid=split_train_test(train,87.5)
  x_train,y_train=split_x_y(train)
  x_val,y_val=split_x_y(valid)
  x_test,y_test=split_x_y(test)
  m,n = x_train.shape
  m_test, n_test = x_test.shape
  mus = np.zeros(m)
  b = 0
  E = np.zeros(m)
  eta = 0
  H = 0
  L = 0
  K = x_train@x_train.T
  iters = 0
  max_iters = iter
  C = c
  while iters < max_iters:
    count_change = 0
    print(iters)
    for i in range(m):
      f = b + np.sum(np.multiply(np.multiply(mus,y_train),K[:,i]))
      E[i] = b + np.sum(np.multiply(np.multiply(mus,y_train),K[:,i])) - y_train[i]
      if ((y_train[i]*E[i]<-bound and mus[i]<C) or (y_train[i]*E[i]>bound and mus[i]>0)):
  
        j = math.ceil((m-1)*np.random.rand())
        while j==i:
          j = math.ceil((m-1)*np.random.rand())
        E[j] = b + np.sum(np.multiply(np.multiply(mus,y_train),K[:,j])) - y_train[j]
        mus_i_old = mus[i]
        mus_j_old = mus[j]
        if y_train[i]==y_train[j]:
          L = max(0.0,(mus[j]+mus[i]-C))
          H = min(C, (mus[j]+mus[i]))
        else:
          L = max(0, mus[j]-mus[i])
          H = min(C, C+mus[j]-mus[i])
        if L==H:
          continue
        eta = 2*K[i][j]-K[i][i]-K[j][j]
        if eta>=0:
          continue
        mus[j] = mus[j] - (y_train[j]*(E[i]-E[j]))/eta
        #print(j,mus[j])
        #print(L,H)
        mus[j] = min(H,mus[j])
        mus[j] = max(L,mus[j])
        #print(j,mus[j])
  
        if (abs(mus[j]-mus_j_old)<bound):
          mus[j] = mus_j_old
          continue
        #print(abs(mus[j]-mus_j_old))
        mus[i] = mus[i] + y_train[i]*y_train[j]*(mus_j_old-mus[j])
        b1 = b -E[i]-y_train[i]*(mus[i]-mus_i_old)*K[i][i]-y_train[j]*(mus[j]-mus_j_old)*K[i][j]
        b2 = b -E[j]-y_train[i]*(mus[i]-mus_i_old)*K[i][j]-y_train[j]*(mus[j]-mus_j_old)*K[j][j]
        if 0<mus[i] and mus[i]<C:
          b = b1
        elif 0<mus[j] and mus[j]<C:
          b = b2
        else:
          b = (b1+b2)/2
        count_change += 1
    if count_change==0:
      iters += 1
      print(mus)
    else:
      iters = 0
    w = 0
    for i in range(m):
      if mus[i]>0:
        w += mus[i]*y_train[i]*x_train[i,:]
  
  y_pred = np.zeros(m_test)
  for i in range(m_test):
    sum = 0
    for j in range(m):
      if mus[j]>=0:
        sum += mus[j]*y_train[j]*np.dot(x_train[j],x_test[i].T)
    sum += b
    y_pred[i] = np.sign(sum)


  return y_pred,y_test

In [ ]:
y_pred,y_test=SVMtrain(data,15,0.01,20)

In [36]:
h_test=np.rint(y_pred).reshape((y_test.shape[0],1))
y_test=y_test.reshape((y_test.shape[0],1))
error = (y_test-np.rint(h_test)).tolist()
# print(error)
som = (y_test+np.rint(h_test)).tolist()
# print(som)
FN = error.count([2.0])
# print(FN)
FP = error.count([-2.0])
# print(FP)
TN = som.count([-2.0])
# print(TN)
TP = som.count([2.0])
# print(TP)
accuracy = (TP+TN)/(TP+TN+FP+FN)
sensitivity = TP/(TP+FN)
specificity = TN/(FP+TN)
conf_matrix = np.array(([TP,FP],[FN,TN]))

In [12]:
print("Accuracy={}".format(accuracy))
print("Sensitivity={}".format(sensitivity))
print("Specificity={}".format(specificity))
print(conf_matrix)

Accuracy=0.813953488372093
Sensitivity=0.9377777777777778
Specificity=0.6780487804878049
[[211  66]
 [ 14 139]]
